####**Character segmentation using contour-based approach.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from os.path import splitext, basename
from keras.models import model_from_json
import glob
import matplotlib
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow
import matplotlib.gridspec as gridspec

Loading the localized license number plates.


In [ ]:
def load_image():
  train_images = []
  image_path ='/content/drive/MyDrive/Colab Notebooks/ALPR/Localized_Nepali_Number_Plates/'
  image = os.listdir(image_path)
  for sample in image:
    img_path =image_path+sample
    train = cv2.imread(img_path)
    train_images.append(train)
  return train_images

Grayscale and binray conversion of the images.

In [ ]:
def imageprocessing(image):

  #Convert to grayscale  image
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  #Applied inversed thresh_binary 
  binary = cv2.threshold(gray,180,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
  
  return binary

Method to save images

In [ ]:
def save_image(epoch,crop_characters):
  for i in range(len(crop_characters)):
    crop_characters[i] = cv2.cvtColor(crop_characters[i], cv2.COLOR_BGR2RGB)

    cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/ALPR/SISR/ALPR_Distorted/Segmented/image{}_{}.jpg'.format(epoch,i),cv2.cvtColor(crop_characters[i],cv2.COLOR_BGR2RGB))

    

Obtaining the segmented characters and saving them.

In [ ]:
# Create sort_contours() function to grab the contour of each digit from left to right
for i in range(len(image)):
  # plt.imshow(image[i])
    binary =imageprocessing(image[i])
  
    def sort_contours(cnts,reverse = False):
        i = 0
        boundingBoxes = [cv2.boundingRect(c) for c in cnts]
        (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
        return cnts


    cont, _  = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # creat a copy version "test_roi" of plat_image to draw bounding box
    test_roi = image[i].copy()

    # Initialize a list which will be used to append charater image
    crop_characters = []

    # define standard width and height of character
    digit_w, digit_h = 30, 50

    for c in sort_contours(cont):

        (x, y, w, h) = cv2.boundingRect(c)
        ratio = h/w
    
        if 0.0<=ratio<=2:# Only select contour with defined ratio
        # print(h/image.shape[0])
            if h/plate_image.shape[0]>=0.2: # Select contour which has the height larger than 20% of the plate
            
            # print(k)
            # Draw bounding box arroung digit number
                cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)

            # Sperate number and gibe prediction
                curr_num = image[i][y:y+h,x:x+w]
                curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
                crop_characters.append(curr_num)
            
        
    save_image(i,crop_characters)

